# FastMap -- Data Mining

In this doc, we need to figure out these points:

1. How it works?
2. Why it works?
3. How to analyze its performance?
4. How to use it in application?
5. What's the comparison results with other state-out-arts?
6. What's the current evolvements of it?

## Intuition of the idea

Designing feature extraction functions can be hard. It is relatively easier for a domain expert to assess the similarity/distance of two objects. Given only the distance information though, it is not obvious how to map objects into points.

And here the goal is to embedding objects into Eclidean space as points which are $k$-dimentional vectors.

### Why choose Euclidean space?

**Enable Spatial access methods (SAMs)** to answer queries more efficiently.

Retrieval of distances: although its more clear to know the similarity of two objects, but it may be computational difficult. And here comes the benefit of embedding at Euclidean space.

Moreover, for visualization and data-mining (unsupervised).

## How it works?

Input: $\mathcal{D} (O_i,O_j)$ a distance/dis-similarity function for every pair of nodes.

Output: the embedding. $P_i=(x_{i,1},x_{i,2},...,x_{i,k})$ for $O_i$

Requirements: $\mathcal{D}()$ is:

- Non-negative
- Symmetric
- Obeys the triangular inequality

## Comparison with benchmark

**Multi-Dimensional Scaling (MDS)** is our benchmark. 

Two ways to evaluate it:

1. Time efficiency or even consider storage efficiency.
2. Preservation of original relations (distance) 